In [2]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

2025-05-20 17:45:14.130796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-20 17:45:14.152008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-20 17:45:14.158294: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-20 17:45:14.245607: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [74]:
DATA_DIR = "images"
IMG_HEIGHT, IMG_WIDTH = 240, 320

file_paths = [os.path.join(DATA_DIR, fname) for fname in os.listdir(DATA_DIR) if fname.endswith(('.jpg', '.png', '.jpeg'))]

# Map label strings to integers
def get_label(filename):
    fname = os.path.basename(filename).lower()
    if fname.startswith('left'):
        return 0
    elif fname.startswith('right'):
        return 1
    elif fname.startswith('no'):
        return 2
    else:
        return -1  # invalid label (skip later)

# Filter out files without valid labels
filtered_files_labels = [(f, get_label(f)) for f in file_paths if get_label(f) != -1]
file_paths, labels = zip(*filtered_files_labels)

def load_and_preprocess_image(path):
    img = image.load_img(path, target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode="grayscale")
    img_array = image.img_to_array(img) / 255.0

    # img_array shape: (240, 320, 1) already because color_mode='grayscale'
    return img_array

images = [load_and_preprocess_image(path) for path in file_paths]

X = np.array(images)  # shape (N, 240, 320, 1)
y = np.array(labels)  # shape (N,)
y = tf.keras.utils.to_categorical(y, num_classes=3)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [75]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.3)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_crop(image, size=[220, 300, 1])  # random crop smaller area
    image = tf.image.resize(image, [240, 320])
    return image, label

# Create dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(buffer_size=1000).map(augment).batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

In [76]:
import tensorflow as tf
from keras.applications import MobileNetV2
from keras.layers import Input, Concatenate, GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
# Input: grayscale image
input_layer = Input(shape=(240, 320, 1))  # grayscale

# Convert grayscale to 3 channels (MobileNetV2 expects RGB)
x = Concatenate()([input_layer, input_layer, input_layer])  # shape becomes (240, 320, 3)

# Load base model (pretrained on ImageNet)
base_model = MobileNetV2(input_shape=(240, 320, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # freeze weights for now

# Apply base model
x = base_model(x, training=False)

# Custom classification head
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(3, activation='softmax')(x)

# Final model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/tmp/ipykernel_115423/2038761873.py:12: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(240, 320, 3), include_top=False, weights='imagenet')


In [77]:
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=test_ds, epochs=5)


Epoch 1/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 8s 58ms/step - accuracy: 0.6036 - loss: 0.9435 - val_accuracy: 0.7933 - val_loss: 0.4816
Epoch 2/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.8338 - loss: 0.4303 - val_accuracy: 0.9017 - val_loss: 0.3017
Epoch 3/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8667 - loss: 0.3570 - val_accuracy: 0.9267 - val_loss: 0.2413
Epoch 4/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9032 - loss: 0.2723 - val_accuracy: 0.9167 - val_loss: 0.2228
Epoch 5/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.9076 - loss: 0.2553 - val_accuracy: 0.9367 - val_loss: 0.1912


In [43]:
#processed_image = load_and_preprocess_image("test.jpg")
#processed_image = np.expand_dims(processed_image, axis=0)
#prediction = model.predict(processed_image)
#print(prediction)


In [79]:
model.save("prawus2.keras")

In [36]:
import random;
model.evaluate(X_test,y_test)
for i in range(100):
    rand=random.randint(0,3000)
    print(file_paths[rand])
    img = load_and_preprocess_image(file_paths[rand])
    img = np.expand_dims(img, axis=-1)  # add channels dim
    img = np.expand_dims(img, axis=0) 
    print(model.predict(img));

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9758 - loss: 0.1140
images/left_2025-05-15_16-38-50_633.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[9.9969494e-01 3.0493288e-04 7.7250156e-08]]
images/right_2025-05-15_16-23-50_123.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[1.7936204e-06 9.9999821e-01 1.3604836e-08]]
images/left_2025-05-15_16-40-35_160.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[9.9999988e-01 2.3561984e-08 1.2942397e-07]]
images/no_2025-05-15_16-45-17_365.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[3.2253056e-03 3.0437361e-05 9.9674428e-01]]
images/right_2025-05-15_16-37-33_879.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[[2.2283226e-05 9.9997640e-01 1.3354580e-06]]
images/no_2025-05-15_16-46-40_883.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[1.7266600e-05 4.8664640e-07 9.9998224e-01]]
images/right_2025-05-15_16-48-11_891.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[1.1733827e-12 1.0000000e+00 2.6167518e-10]]
images/left_2025-05-15_16-39-55_612.jpg
1/1 ━━━━━━━━━━━━━━━

In [33]:
DATA_DIR = "images2"
file_paths = [os.path.join(DATA_DIR, fname) for fname in os.listdir(DATA_DIR) if fname.endswith(('.jpg', '.png', '.jpeg'))]
for i in range(100):
    print(file_paths[i])
    img = image.load_img(file_paths[i], target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode="grayscale")
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=-1) 
    img_array = np.expand_dims(img_array, axis=0)
    print(model.predict(img_array));

images2/left_2025-05-20_00-05-06_954.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.00847351 0.8824692  0.10905731]]
images2/left_2025-05-20_00-05-07_703.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[0.11851753 0.75771815 0.12376428]]
images2/left_2025-05-20_00-05-08_216.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[0.03120819 0.8777033  0.0910885 ]]
images2/left_2025-05-20_00-05-08_517.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[[0.0731079  0.64374876 0.28314328]]
images2/left_2025-05-20_00-05-09_062.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[0.24100922 0.66283137 0.09615942]]
images2/left_2025-05-20_00-05-09_323.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[0.30885163 0.6053696  0.08577874]]
images2/left_2025-05-20_00-05-09_814.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
[[0.23546687 0.6702372  0.09429596]]
images2/left_2025-05-20_00-05-10_124.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[0.06613081 0.76927835 0.1645909 ]]
images2/left_2025-05-20_00-05-10_564.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/

In [ ]:
for x, y in train_ds.take(1):
    print("X shape:", x.shape)
    print("Y shape:", y.shape)
    print("Sample labels:", y.numpy())

In [69]:
for x_batch, y_batch in test_ds.take(1):
    preds = model(x_batch, training=False)
    print("First prediction logits:", preds[0].numpy())
    print("Predicted class:", tf.argmax(preds[0]).numpy())
    print("True class:", y_batch[0].numpy())

First prediction logits: [0. 0. 1.]
Predicted class: 2
True class: 0


In [ ]:
base_model.trainable = True


for layer in base_model.layers[:-20]:
    layer.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data=test_ds, epochs=5)

Epoch 1/2
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 63ms/step - accuracy: 0.7346 - loss: 1.0910 - val_accuracy: 0.8983 - val_loss: 0.2660
Epoch 2/2
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9564 - loss: 0.1275 - val_accuracy: 0.9267 - val_loss: 0.2118


In [ ]:
base_model.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#model.fit(train_ds, validation_data=test_ds, epochs=4)

Epoch 1/4
75/75 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - accuracy: 0.9483 - loss: 0.1365 - val_accuracy: 0.9500 - val_loss: 0.1245
Epoch 2/4
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9679 - loss: 0.0908 - val_accuracy: 0.9600 - val_loss: 0.1078
Epoch 3/4
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.9764 - loss: 0.0787 - val_accuracy: 0.9650 - val_loss: 0.1047
Epoch 4/4
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.9816 - loss: 0.0681 - val_accuracy: 0.9667 - val_loss: 0.0953
